# Data Cleaning

In [ ]:
cols = [0, 1, 3, 4, 5,6, 7, 8, 14, 15, 16, 17, 18, 19, 22, 30, 31, 32, 34, 35, 36, 41, 42, 43, 45, 51, 52, 53, 54, 55, 56, 58, 63, 64, 65, 67, 73, 74, 75, 76, 77]

data = data[data.columns.values[cols]]

data_en = data[data['lang'] == 'en'].reset_index(drop=True)

# RT flag

data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

# Date check

data_en['Date_Flag'] = data_en['created_at'].str[:10] == 'Tue Aug 18'

data_en = data_en[np.where((data_en['Date_Flag'] == True),True,False)].reset_index(drop=True)

# Tree

############### is_quote_tweet --> False --> Non RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']==False),True,False)].reset_index(drop=True)

interim['extended_tweet_full_text'] = np.where(~interim['truncated'], interim['text'], interim['extended_tweet_full_text'])

non_rep = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

############### is_quote_tweet --> False --> RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

############### is_quote_tweet --> True --> Non RT ###############

interim = data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

interim['extended_tweet_full_text'] = np.where(~interim['truncated'],interim['text'],interim['extended_tweet_full_text'])

interim = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]

interim.columns = non_rep.columns

interim = interim.drop_duplicates(['id']).reset_index(drop=True)

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

interim = interim[['created_at','QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

############### is_quote_tweet --> True --> RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

interim = data_en[np.where((data_en['is_quote_tweet']== True ) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

interim = interim[['created_at', 'QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

#####  1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [ ]:
cleaned_data = non_rep.copy()

cleaned_data['extended_tweet_full_text_duplicate'] = cleaned_data['extended_tweet_full_text']

#Convert to lower case
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.lower()

#Removing emojis
def demoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
                               "]+", flags=re.UNICODE)
    return(emoji_pattern.sub(r'', text))

cleaned_data[u'extended_tweet_full_text'] = cleaned_data[u'extended_tweet_full_text'].astype(str)
cleaned_data[u'extended_tweet_full_text'] = cleaned_data[u'extended_tweet_full_text'].apply(lambda x:demoji(x))

#Remove URLs
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

#Remove user @
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r'\@[\w]+', "", regex=True)

#Remove punctuations
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))

#More Cleaning
cleaned_data['extended_tweet_full_text']=cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

#Tokenizing
cleaned_data.extended_tweet_full_text = cleaned_data.extended_tweet_full_text.astype(str)
cleaned_data['tokenized_extended_tweet_full_text'] = cleaned_data.apply(lambda row: nltk.word_tokenize(row.extended_tweet_full_text), axis=1)

# remove stopwords
cleaned_data['filtered_extended_tweet_full_text'] = cleaned_data['tokenized_extended_tweet_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

#Stemming
ps = PorterStemmer()
cleaned_data['stemmed_extended_tweet_full_text'] = cleaned_data['filtered_extended_tweet_full_text'].apply(lambda x: [ps.stem(y) for y in x])

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#Lemmatizing
lemmatizer = WordNetLemmatizer()
cleaned_data['lemmatized_extended_tweet_full_text'] = cleaned_data['filtered_extended_tweet_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

#Joining the lemmetized tokens to form string
cleaned_data['final'] = cleaned_data['lemmatized_extended_tweet_full_text'].apply(lambda x: " ".join([word for word in x]))

#Remove punctuations
cleaned_data['final'] = cleaned_data['final'].str.translate(str.maketrans("", "", string.punctuation))
cleaned_data['final'] = cleaned_data['final'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
cleaned_data['extended_tweet_full_text']=cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

cols_2 = [0, 1, 2, 3 ,5, 10]
data_final = cleaned_data[cleaned_data.columns.values[cols_2]]


data_final.to_csv('/home/manikya_varshney/Documents/Python/Yale/Final Data/final(Cleaned and Analysed)_August18th.csv',index=False)

data_final.to_parquet('/home/manikya_varshney/Documents/Python/Yale/Final Data/final(Cleaned and Analysed via gzip)_August18th.gzip', compression='gzip')

